# UCU 2024 - Proyecto 1
Web Scraping - Proyecto para la asignatura Programación para Análisis de Datos

In [2]:
import requests
from bs4 import BeautifulSoup
import json

Set constants

In [4]:
URL = "https://www.infocasas.com.uy/venta/inmuebles"
CITIES = ("montevideo", "maldonado", "canelones")

Get info

In [54]:
properties = {
  "ciudades": []
}

for city in CITIES:
    city_info = {
      "nombre": city.title(),
      "propiedades": []
    }

    req = requests.get(f"{URL}/{city}", headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})
    soup = BeautifulSoup(req.text)
    properties_html = soup.find_all("div", {"class": "lc-dataWrapper"})
    for property in properties_html:
        price = property.find("div", class_="lc-price").text or "U$S 0"

        info = property.find("div", class_="lc-typologyTag").findAll("strong")
        tamano = next((x.text for x in info if "m²" in x.text), "0 m²")
        habitaciones = next((x.text for x in info if "Dorm" in x.text), "0 Dorm.")

        link = property.find("a")['href']

        try:
          property_info = {
            "precio": price.split()[-1],
            "tamano": int(tamano.split()[0]),
            "habitaciones": int(habitaciones.split()[0]),
            "link": f"https://www.infocasas.com.uy{link}"
          }
          city_info["propiedades"].append(property_info)
        except Exception as e:
          print(f"Error parsing property: {e.__cause__}")


    properties["ciudades"].append(city_info)

And create the json file

In [55]:
with open("propiedades.json", "w") as json_file:
    json_file.write(json.dumps(properties, indent=3))